In [ ]:
# Import packages
import pandas as pd
import datetime as dt
import statistics as stat
pd.set_option('display.max_rows', 800)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Scheduled Stats

In [ ]:
# Import Scheduled Stats for Neurology providers
specialty = "Neurology"
stats=pd.read_csv(f"{specialty}/Neuro_data (1).csv")
# Select schedulable hours and linking fields
st=stats[['SCHEDULABLE_HRS','PROV_NAME','STATISTICS_DATE(FY)']]
# Rename selected columns
st=st.rename({'PROV_NAME': "Provider Name", 'STATISTICS_DATE(FY)': 'Appointment Date'}, axis=1)
# Convert Object to date format
st['Appointment Date']=pd.to_datetime(st['Appointment Date']).dt.date
# Group dataframe by provider name and appointment date
st=st.groupby(['Provider Name','Appointment Date'],as_index=False).count()
st.head()

## Encounter

In [ ]:
# Import Encounter Data
encounter = pd.read_excel(f"{specialty}/neurology_2019 (1).xlsx").rename(columns={
    'Clinic___': "Clinic",
    'Appointment Cancel Date (DAY)':'Appointment Cancel Date',
    'Appointment Date (DAY)': 'Appointment Date',
    'Appointment Made Date (DAY)': 'Appointment Made Date',
    'Provider Name_': 'Provider Name',
    '*Expected: (AGG)': "*Expected:",
    '<=24 Hours: (Cancelled) (SUM)': "<=24 Hours: (Cancelled)",
    'Cancelled: (SUM)': "Cancelled:",
    'No Show: (SUM)': "No Show:",
    'Rescheduled: (SUM)': "Rescheduled:",
    'Total Attended (SUM)': "Total Attended"
}).drop('Encounter CSN ID', axis=1)
# Import Provider List
providers =pd.read_csv(f"{'Neurology'}/providers.csv")
# Merge two datasets
df2=encounter.merge(providers, how="inner", on="Provider Name")
# Combine Epilepsy with General
df2['SubSpecialty']=df2['SubSpecialty'].replace(['Neurology: Epilepsy'],'Neurology: General')
# Remove NP/PA
df2=df2[df2['SubSpecialty']!='Neurology - NP / PA']
step_1=df2[["Rescheduled:","*Expected:","Total Attended",'SubSpecialty','Appointment Date','Appointment Made Date',"No Show:",'Appointment Cancel Date',"Cancelled:","<=24 Hours: (Cancelled)",'Visit Type','Provider Name']]

In [ ]:
# Define New Patients as Visit Type of NEW and MYC VIDEO VISIT NEW
for row in step_1.iterrows():
    if row[1]['Visit Type'] in ['NEW', 'MYC VIDEO VISIT NEW']:
        step_1.at[row[0], 'NEW'] = True
    else:
        step_1.at[row[0], 'NEW'] = False
step_1['NEW'] = step_1['NEW'].astype(int)
step_1=step_1[step_1['NEW']==1]

In [ ]:
# Convert Object to date formats
step_1['Appointment Date']=pd.to_datetime(step_1['Appointment Date'])
step_1['Appointment Made Date']=pd.to_datetime(step_1['Appointment Made Date'])
step_1['Appointment Made Date']=step_1['Appointment Made Date'].dt.date
step_1['Appointment Date']=step_1['Appointment Date'].dt.date
step_1['Appointment Cancel Date']=pd.to_datetime(step_1['Appointment Cancel Date']).dt.date
step_1['year_month_contact']=pd.to_datetime(step_1['Appointment Date']).dt.strftime('%Y-%m')
step_1['week_contact']=pd.to_datetime(step_1['Appointment Date']).dt.strftime('%Y-%U')
step_1['year_month_appmade']=pd.to_datetime(step_1['Appointment Made Date']).dt.strftime('%Y-%m')
step_1['week_appmade']=pd.to_datetime(step_1['Appointment Made Date']).dt.strftime('%Y-%U')
step_1['Canceled before contact date']=(step_1['Appointment Cancel Date'] < step_1['Appointment Date'])*1

In [ ]:
# Merge encounter with scheduled stats
step_1=pd.merge(step_1,st,how="inner", on=['Provider Name','Appointment Date'])

In [ ]:
# Segregate each subspecialty of Neurology as a dataframe
ncbc=step_1[step_1['Canceled before contact date']==0]
ss1=ncbc[ncbc['SubSpecialty']=='Neurology: General']
ss2=ncbc[ncbc['SubSpecialty']=='Neurology: Movement Disorders']
ss3=ncbc[ncbc['SubSpecialty']=='Neurology: Pediatric ']
ss4=ncbc[ncbc['SubSpecialty']=='Neurology: Cerebrovascular Disease & Stroke']
ss5=ncbc[ncbc['SubSpecialty']=='Neuropsychology']
ss6=ncbc[ncbc['SubSpecialty']=='Neurology: Alzheimers / Dementia']

## Queue Length

In [ ]:
# Calculate Queue Length
list_df=[ss1,ss2,ss3,ss4,ss5,ss6]
for i in list_df:
    date_list_contact=i['Appointment Date']
    queue_length=[]
    for x in date_list_contact:
        num=i["*Expected:"][(i['Appointment Date']>x)&(i['Appointment Made Date']<=x)].count()
        queue_length.append(num)
    i['queue_length']=queue_length

## Wait Times

In [ ]:
# Calculate Wait Times
ncbc=pd.concat(list_df,axis=0)
ncbc['wait day']=ncbc['Appointment Date']-ncbc['Appointment Made Date']
ncbc['wait day']=ncbc['wait day'].dt.days.astype(int)
# Limit Time Period
ncbc_period=ncbc[(ncbc['Appointment Date']>=dt.date(2019, 1, 1))&(ncbc['Appointment Date']<=dt.date(2019, 12, 31))]
# Calculate average Wait Times per week
wq_week=ncbc_period.groupby(['SubSpecialty','week_appmade'],as_index=False).agg(
    {'wait day': 'mean','queue_length':'mean'})
wq_week.rename(columns = {'week_appmade':'Week'}, inplace = True) 

## Demand

In [ ]:
# Limit Period to 2019 
contact=step_1[(step_1['Appointment Date']>=dt.date(2019, 1, 1))&(step_1['Appointment Date']<=dt.date(2019, 12, 31))]
# Demand is calculated based on appointment made date
demand_week=contact.groupby(['SubSpecialty','week_appmade'],as_index=False).agg(
    {'Canceled before contact date':'sum',"*Expected:":'count'})
# Demand= # of expected visits - # of early cancelation
demand_week['demand']=demand_week["*Expected:"]-demand_week['Canceled before contact date']
demand_week=demand_week[['week_appmade','demand','SubSpecialty']]

## Supply

In [ ]:
# Supply is calculated based on appointment/contact date
# Supply is schedulable hours for regression purpose 
supply_week=contact.groupby(['SubSpecialty','week_contact'],as_index=False).agg(
    {'SCHEDULABLE_HRS':'sum'})

## Combine demand, supply, and wait times  in one dataframe

In [ ]:
# Combine demand and supply
supply_week.rename(columns = {'week_contact':'Week'}, inplace = True) 
demand_week.rename(columns = {'week_appmade':'Week'}, inplace = True) 
week_d_s=pd.merge(supply_week,demand_week,how='outer',on=['SubSpecialty', 'Week'])
week_d_s=week_d_s.fillna(0)
# Combine demand, supply, and wait times in one dataframe
week_all=pd.merge(wq_week,week_d_s,how='outer',on=['SubSpecialty', 'Week'])
week_all=week_all.fillna(0)

## Regression

In [ ]:
AIC=[]
p_value_reg=pd.DataFrame() 
coef_reg=pd.DataFrame() 
r_squared=[]
elasticity=pd.DataFrame() 

import statsmodels.api as sm
for i in week_all['SubSpecialty'].unique():
    df= week_all[week_all['SubSpecialty']==i]
    y= df['wait day'].values.reshape(-1,1)[1:-2]
    # Convert the wait days to hours
    y=24*y

## Create random walk for demand 
    y_=df['demand'].values[1:]
    x_=df['demand'].shift(1)[1:]
    x_ = sm.add_constant(x_)
    mod_=sm.OLS(y_,x_).fit()
    pred=mod_.predict(x_)
    
## X    
    x=pd.DataFrame(df['SCHEDULABLE_HRS'][1:-2])
#     x['lag wait time']= df['wait day'].shift(1)[1:-2]
    x['supply foward 1']= df['SCHEDULABLE_HRS'].shift(-1)[1:-2]
    x['pred demand']=pred[:-2]
    x['supply forward 2']= df['SCHEDULABLE_HRS'].shift(-2)[1:-2]
    x['intercept']=1
    name=x.columns
    
    x=x.values.reshape(-5,5)
    mod = sm.OLS(y,x)
    fii = mod.fit()
    p_values = fii.summary2().tables[1]['P>|t|'].values.round(5)
    R_Squared=fii.summary2().tables[0].iloc[6,1]
    Coeff= fii.summary2().tables[1]['Coef.'].values.round(3)
    els=Coeff*x.mean(axis=0)/y.mean()
    aic=fii.summary2().tables[0].iloc[1,3]
    p_value_reg[i]= p_values
    coef_reg[i]= Coeff
    elasticity[i]=els
    r_squared.append(R_Squared)
    AIC.append(aic)
    
p_value_reg.index = name
coef_reg.index= name
elasticity.index= name
r_squared=pd.DataFrame(r_squared) 
r_squared.index=coef_reg.columns.values

In [ ]:
p_value_reg

In [ ]:
coef_reg

In [ ]:
elasticity

In [ ]:
r_squared

In [ ]:
AIC

## Movement Disorders (schedulable hours significant)

In [ ]:
df= week_all[week_all['SubSpecialty']=='Neurology: Movement Disorders']
y= df['wait day'].values.reshape(-1,1)[1:-2]
y=24*y

y_=df['demand'].values[1:]
x_=df['demand'].shift(1)[1:]
x_ = sm.add_constant(x_)
mod_=sm.OLS(y_,x_).fit()
pred=mod_.predict(x_)
    
    
x=pd.DataFrame(df['SCHEDULABLE_HRS'][1:-2])
x['supply foward 1']= df['SCHEDULABLE_HRS'].shift(-1)[1:-2]
x['pred demand']=pred[:-2]
x['supply forward 2']= df['SCHEDULABLE_HRS'].shift(-2)[1:-2]
x['intercept']=1

x=x.values.reshape(-5,5)
mod = sm.OLS(y,x)
fii = mod.fit()

## Evaluate 
import pandas as pd
slope, intercept = np.polyfit(fii.predict(x)/24,y/24, 1)
# Create a list of values in the best fit line
abline_values = [slope * i + intercept for i in fii.predict(x)/24]

plt.figure(figsize=(8, 8))
plt.plot(fii.predict(x)/24,y/24,'o',c='#ff7f0e')
plt.plot(fii.predict(x)/24, abline_values, 'b')
plt.title('Prediction of Wait Times(Days)')  
plt.xlabel('Predicted Wait Times (Days)')  
plt.ylabel('Actual Wait Times (Days)')  
plt.show()

In [ ]:
## Actual weekly average wait time= 51 days
## Target wait time =31 days
reduced=[]
for i in range(10,200,5):
    i=i/100
    actual=week_all[week_all['SubSpecialty']=='Neurology: Movement Disorders']['wait day'].mean()*24
    reduced_wt= actual*(1+elasticity['Neurology: Movement Disorders'][0]*i)
    reduced.append(reduced_wt/24)

In [ ]:
import pandas as pd
plt.figure(figsize=(8, 8))
plt.plot([element *(19/100) for element in range(110,300,5)],reduced,'o')
plt.title('Decrease of Wait Times with Increase in Schedulable Hours')  
plt.xlabel('Weekly Schedulable Hours')  
plt.ylabel('New Patient Wait Times')  
import matplotlib.pyplot as plt
plt.axhline(y = 31, color = 'r', linestyle = '-')
plt.show()

## Random Walk for Demand

In [ ]:
## Evaluate random walk for demand    
import statsmodels.api as sm
for i in week_all['SubSpecialty'].unique():
    df= week_all[week_all['SubSpecialty']==i]
    y_=df['demand'].values[1:]
    x_=df['demand'].shift(1)[1:]
    mod_=sm.OLS(y_,x_).fit()
    pred=mod_.predict(x_)
    
    plt.plot(df['Week'][1:],y_,'o') ## actual demand
    plt.plot(df['Week'][1:],pred,'x') ## predicted demand
    plt.title(i)  
    plt.xlabel('week')  
    plt.ylabel('demand')  
    plt.show()